In [14]:
# Installing selenium 
pip install selenium


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
# Importing the Libraries 
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import time

from bs4 import BeautifulSoup as soupify
from selenium.webdriver.support.wait import WebDriverWait
from tqdm import tqdm
import re


In [32]:
# Creating a driver 
driver = webdriver.Firefox(executable_path=r'"C:\Users\abcd\Downloads\geckodriver-v0.32.2-win-aarch64\geckodriver.exe"')
driver.get("https://pib.gov.in/PMContents/PMContents.aspx?menuid=2&Lang=1&RegionId=3")

C:\Users\abcd\AppData\Local\Temp\ipykernel_22956\3887639555.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=r'"C:\Users\abcd\Downloads\geckodriver-v0.32.2-win-aarch64\geckodriver.exe"')


In [33]:
#Using HTML code (inspect option), we extracted the required code for Day, Month, Year located on the file
# Select function is being used to select the dropdown options for the year, month, and day on the website
date_form_day = Select(driver.find_element(By.ID, "ContentPlaceHolder1_ddlday"))
date_form_month = Select(driver.find_element(By.ID, "ContentPlaceHolder1_ddlMonth"))
date_form_year = Select(driver.find_element(By.ID, "ContentPlaceHolder1_ddlYear"))

# Storing the value of months
month =[]
i= 1
for option in date_form_month.options:
    month.append(i)
    i+=1

# Storing the value of years
year =[]
for option in date_form_year.options:
    year.append(int(option.text))

print(month)
print(year)


GETTING URLS OF SPEECHES

In [37]:
# Storing the URLs of all the speeches in a list 
t=[]

# Iterate through the years using tqdm progress bar
for i in tqdm(year):
    # Iterate through the months
    for j in range(len(month)):
        # Select the year dropdown option on the website
        date_form_year = Select(driver.find_element(By.ID, "ContentPlaceHolder1_ddlYear"))
        y = str(i)
        date_form_year.select_by_value(y)
        # Wait for the website to load
        WebDriverWait(driver,5)
         # Select the month dropdown option on the website
        date_form_month = Select(driver.find_element(By.ID, "ContentPlaceHolder1_ddlMonth"))
        m = str(month[j])
        date_form_month.select_by_value(m)
        WebDriverWait(driver,5)
        # Wait for the website to load

        # Parse the HTML source code of the website using BeautifulSoup
        doc = soupify(driver.page_source)

        # Extract the URLs for speeches on the website
        speeches = doc.select(".num > li > a")
        for k in range(len(speeches)):
            tar_speech = speeches[k]
            tar_speech.attrs['href']
            BASE_URL = "https://pib.gov.in"
            SPEECH_URL = tar_speech.attrs['href']
            TARGET_URL = BASE_URL + SPEECH_URL 
            t.append(TARGET_URL) 
            
        # Select the day, month, and year dropdown options again (not clear why this is done)
        date_form_day = Select(driver.find_element(By.ID, "ContentPlaceHolder1_ddlday"))
        date_form_month = Select(driver.find_element(By.ID, "ContentPlaceHolder1_ddlMonth"))
        date_form_year = Select(driver.find_element(By.ID, "ContentPlaceHolder1_ddlYear"))
            
        WebDriverWait(driver,5)
    

print(len(t))

100%|██████████| 7/7 [01:12<00:00, 10.30s/it]


In [40]:

#if set t < lenght of t
duplicates = set([x for x in t if t.count(x) > 1])

if len(duplicates) > 0:
    print(f'Duplicate strings found: {duplicates}')
else:
    print('No duplicate strings found')


No duplicate strings found


In [41]:
len(duplicates)

0

In [ ]:

data = {
    'Date' : [],
    'Title' : [],
    'Location' : [],
    'Speech': []
}


Extract the Text, Date, Location and Title of all speeches

ASK TARANG 

In [58]:
# Create a dictionary to store the extracted data
data = {
    'Date' : [],
    'Title' : [],
    'Location' : [],
    'Speech': []
}

# Loop through each URL in the list t, 
# starting from index 719 (we have scrapped speeches in two batches)
for target in tqdm(t[719:]):
    # Initialize an empty list to store the speech paragraphs
    speech_list = []
    # Load the webpage for the current URL using Selenium and 
    # parse the HTML source code using BeautifulSoup
    driver.get(target)
    speech_doc = soupify(driver.page_source)

    # Extract the speech URL from the first iframe element on the page, if it exists
    iframe_list = speech_doc.select("iframe")
    if iframe_list:
        NEW_SPEECH_URL = BASE_URL + '/' + iframe_list[0].attrs['src']

        # Load the webpage for the speech URL and parse the HTML source code using BeautifulSoup
        driver.get(NEW_SPEECH_URL)
        tar_doc = soupify(driver.page_source)

        # Extract the speech paragraphs and store them in the 'Speech' field of the data dictionary
        speech_paras = tar_doc.select("p")
        for para in speech_paras:
            speech_list.append(para.text)
        data_1['Speech'].append(''.join(speech_list))
        
        # Extract the speech title and store it in the 'Title' field of the data dictionary
        title = tar_doc.select("h2")
        data_1['Title'].append(title[0].text.strip('\n').strip())

        # Extract the speech date and store it in the 'Date' field of the data dictionary
        date_text = tar_doc.find_all("div", {'class':'ReleaseDateSubHeaddateTime text-center pt20'})
        date = date_text[0].text
        data_1['Date'].append(re.search(r'\d{1,2}\s+\w{3}\s+\d{4}', date)[0])
        
        # Extract the location mentioned in the speech title, if any, and store it in the 'Location' field of the data dictionary
        text = title[0].text.strip('\n').strip()
        doc = nlp(text)
        for ent in doc.ents:
            found_location = False
            if ent.label == 384:
                data_1['Location'].append(str(ent))
                found_location = True # Set flag to True if a location is found
                break 
        if not found_location: # If no location entity was found, append '.'
            data_1['Location'].append('.')  
    else:
        print("No iframe element found on page:", target)

    

100%|██████████| 550/550 [23:23<00:00,  2.55s/it]


In [46]:
# Check whether we have same length of data
print(len(data['Date']))
print(len(data['Location']))
print(len(data['Speech']))
print(len(data['Title']))

718
715
718
718


In [59]:
# Check whether we have same length of data
print(len(data_1['Date']))
print(len(data_1['Location']))
print(len(data_1['Speech']))
print(len(data_1['Title']))

550
548
550
550


In [48]:
data['Location']

['.',
 'Kanyakumari',
 'Ahmedabad',
 'Gandhinagar',
 'Gujarat',
 'Tirupati',
 '.',
 '.',
 'Kerala',
 'Jagadguru',
 'Jagadguru',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 'Swachh',
 'Swachh',
 'Dahej',
 'Yogoda',
 'Bharuch',
 '.',
 'Shimla Delhi',
 'Surat',
 'PM',
 'PM',
 'Australia',
 '.',
 'Bangladesh',
 'Jharkhand',
 '.',
 '.',
 '.',
 'Malaysia',
 '.',
 '.',
 '.',
 '.',
 '.',
 'Gandhidham',
 'PM',
 '.',
 'Dickoya',
 'Colombo',
 '.',
 'Sangha',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 'Ahmedabad',
 'Yojana',
 'Rajkot',
 'Ahmedabad',
 'Netherlands',
 'The Hague',
 'The Hague',
 'Washington DC',
 'Washington DC',
 'Washington DC',
 '.',
 '.',
 'Lisbon',
 'PM',
 'Yoga-2017',
 'Lucknow',
 '.',
 '.',
 '.',
 'Astana',
 'PM',
 'PM',
 '.',
 '.',
 'Rameswaram',
 '.',
 '.',
 'Israel',
 'Israel',
 'Israel',
 'Israel',
 'Tel Aviv',
 '.',
 'New Delhi',
 'Delhi',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 'Shahanshahpur',
 'PM',
 'Varanasi',
 '.',
 'Amreli',
 'Japan

In [1]:
#the above exercise was done to understand the data limitation. Not every speech title includes location of speech

In [50]:
# Converting the Data into DataFrame 
import pandas as pd 
df_data = pd.DataFrame(data, columns=['Date', 'Speech', 'Title'])
df_data

,Date,Speech,Title
0,31 JAN 2017,2017 में संसद का आज सत्र आरंभ हो रहा है। राष्‍...,Text of PMs statement ahead of Budget Session ...
1,13 JAN 2017,"Most respected Morari Bapu Ji, President of th...",Text of PMs address to the inaugural event of ...
2,09 JAN 2017,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...
3,09 JAN 2017,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...
4,09 JAN 2017,"प्यारे भाईयों और बहनों,हमारे देश में रेलवे,...","Text of PM’s address at ""Bhumi Pujan"" for rede..."
...,...,...,...
713,28 MAR 2021,"My dear countrymen, Namaskar. This time when I...",English rendering of PM’s address in the 75thE...
714,27 MAR 2021,DISCLAIMER: This is the approximate translatio...,English rendering of Prime Minister's address ...
715,26 MAR 2021,"Namaskar!\nExcellencies,\n\nPresident of Bangl...",English rendering of Prime Minister Shri Naren...
716,22 MAR 2021,"My colleague in the Union Cabinet, Shri Gajend...",English rendering of PM’s speech at launch of ...


In [60]:
df_data_1 = pd.DataFrame(data_1, columns=['Date', 'Speech', 'Title'])
df_data_1

,Date,Speech,Title
0,16 MAR 2021,"Excellency,Namaskar!Many thanks for your remar...",English rendering of Prime Minister’s Opening ...
1,12 MAR 2021,"नमस्कार!My colleagues in the Ministry, Kiren R...",Text of PM's address at Global Ayurveda Festival
2,12 MAR 2021,"Present on the dais, Governor of Gujarat Shri ...",English rendering of PM’s address at the inaug...
3,12 MAR 2021,"Your Excellencies,\nPresident Biden,\nPrime Mi...",Prime Minister's opening remarks at the first ...
4,11 MAR 2021,"Distinguished guests,Friends!Vanakkam!This is ...",Text of PM’s address at the launch of e-versio...
...,...,...,...
545,08 APR 2023,Bharat Mata Ki Jai!\n\nBharat Mata Ki Jai!\n\n...,English rendering of PM’s address at launch of...
546,04 APR 2023,ExcellenciesHonourable Minister Mr. Harbers;Sp...,"Text of Address by Dr. P.K. Mishra, Principal ..."
547,04 APR 2023,"Namaskar!Excellencies, heads of state, Academi...",Text of PM’s remarks at International Conferen...
548,03 APR 2023,My colleague in the Union Cabinet Dr. Jitendra...,English rendering of PM’S address at the diamo...


In [2]:
#since we have scrapped speeches in two batches, we create two dataframes then append it

In [51]:
# Saving the DataFrame to .csv file 
df_data.to_csv(r"C:\Users\abcd\Documents\PM_Modi_Speech_Text.csv")

In [61]:
# Saving the DataFrame to .csv file 
df_data_1.to_csv(r"C:\Users\abcd\Documents\PM_Modi_Speech_Text_1.csv")

In [64]:
import pandas as pd

d1 = "C:/Users/abcd/Documents/PM_Modi_Speech_Text.csv"
d2 = "C:/Users/abcd/Documents/PM_Modi_Speech_Text_1.csv"

# Read the CSV file into a dataframe
df1 = pd.read_csv(d1)
df2 = pd.read_csv(d2)


In [66]:
df = pd.concat([df1, df2], ignore_index=True)

In [69]:
df = df.drop('Unnamed: 0', axis=1)

In [75]:
df

,Date,Speech,Title
0,31 JAN 2017,2017 में संसद का आज सत्र आरंभ हो रहा है। राष्‍...,Text of PMs statement ahead of Budget Session ...
1,13 JAN 2017,"Most respected Morari Bapu Ji, President of th...",Text of PMs address to the inaugural event of ...
2,09 JAN 2017,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...
3,09 JAN 2017,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...
4,09 JAN 2017,"प्यारे भाईयों और बहनों,हमारे देश में रेलवे,...","Text of PM’s address at ""Bhumi Pujan"" for rede..."
...,...,...,...
1263,08 APR 2023,Bharat Mata Ki Jai!\n\nBharat Mata Ki Jai!\n\n...,English rendering of PM’s address at launch of...
1264,04 APR 2023,ExcellenciesHonourable Minister Mr. Harbers;Sp...,"Text of Address by Dr. P.K. Mishra, Principal ..."
1265,04 APR 2023,"Namaskar!Excellencies, heads of state, Academi...",Text of PM’s remarks at International Conferen...
1266,03 APR 2023,My colleague in the Union Cabinet Dr. Jitendra...,English rendering of PM’S address at the diamo...


In [71]:
df.to_csv(r"C:\Users\abcd\Documents\PM_Modi_Speech_Text_final.csv")

REMOVAL OF HINDI SPEECHES FROM DATAFRAME

In [97]:
pip install langdetect

     -------------------------------------- 981.5/981.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for langdetect: started
  Running setup.py install for langdetect: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: langdetect is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [99]:
#Import the necessary libraries
from langdetect import detect
import pandas as pd


# Function to detect language of a given text using langdetect library
def detect_language(text):
    try:
        return detect(text)
    except:
        return None

# Apply language detection function to the "speech" column and create a new column "language"
df["language"] = df["Speech"].apply(detect_language)

# Filter the dataframe to keep only rows with English speeches
df_english = df[df["language"] == "en"]

# Drop the "language" column if it's not needed
df_english = df_english.drop("language", axis=1)

# Output the dataframe with only English speeches
print(df_english)


             Date                                             Speech   
2     09 JAN 2017  Chief Minister of Gujarat Shri Vijay Rupani ji...  \
3     09 JAN 2017  I  am delighted to be here at Gift City to ina...   
5     03 JAN 2017  Governor of Andhra Pradesh, Shri E. S. L. Nara...   
6     26 FEB 2017  My dear countrymen, Namaskar. Winter is on its...   
8     21 FEB 2017  Namaskaram. Greetings to everyone. Swami Nirvi...   
...           ...                                                ...   
1263  08 APR 2023  Bharat Mata Ki Jai!\n\nBharat Mata Ki Jai!\n\n...   
1264  04 APR 2023  ExcellenciesHonourable Minister Mr. Harbers;Sp...   
1265  04 APR 2023  Namaskar!Excellencies, heads of state, Academi...   
1266  03 APR 2023  My colleague in the Union Cabinet Dr. Jitendra...   
1267  01 APR 2023  Bharat Mata Ki Jai!Bharat Mata Ki Jai!The Gove...   

                                                  Title answer  
2     Text of PM's address at the Inauguration of No...         
3    

In [103]:
# Export the dataframe with only English speeches to a CSV file
df_english.to_csv(r"C:\Users\abcd\Documents\PM_Modi_Speech_Text_english.csv")

In [101]:
df_english = df_english.drop('answer', axis=1)

FINAL DATAFRAME OF SCRAPPED SPEECH

In [102]:
df_english

,Date,Speech,Title
2,09 JAN 2017,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...
3,09 JAN 2017,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...
5,03 JAN 2017,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...
6,26 FEB 2017,"My dear countrymen, Namaskar. Winter is on its...",English Translation of the text of PM’s ‘Mann ...
8,21 FEB 2017,Namaskaram. Greetings to everyone. Swami Nirvi...,Text of PM’s inaugural address (via video conf...
...,...,...,...
1263,08 APR 2023,Bharat Mata Ki Jai!\n\nBharat Mata Ki Jai!\n\n...,English rendering of PM’s address at launch of...
1264,04 APR 2023,ExcellenciesHonourable Minister Mr. Harbers;Sp...,"Text of Address by Dr. P.K. Mishra, Principal ..."
1265,04 APR 2023,"Namaskar!Excellencies, heads of state, Academi...",Text of PM’s remarks at International Conferen...
1266,03 APR 2023,My colleague in the Union Cabinet Dr. Jitendra...,English rendering of PM’S address at the diamo...
